In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import matplotlib.ticker as ticker

LAYOUT = 'horizontal'
FIGURE_SIZE = (18, 6) if LAYOUT == 'horizontal' else (10, 16)
SAVE_DPI = 1500
AXIS_LABEL_FONTSIZE = 18
TICK_LABEL_FONTSIZE = 18
LEGEND_FONTSIZE = 16
PANEL_LABEL_FONTSIZE = 22
FITTED_CURVE_LINEWIDTH = 2.5
MARKER_SIZE = 8
ERROR_BAR_CAPSIZE = 5
ERROR_BAR_THICKNESS = 1
PANEL_LABEL_X_OFFSET = -0.20
PANEL_LABEL_Y_OFFSET = 1.05
OUTPUT_PNG = 'ic50_combined_panels.png'
OUTPUT_PDF = 'ic50_combined_panels.pdf'

def logistic(x, L, k, x0):
    return L / (1 + np.exp(-k * (np.log10(x) - np.log10(x0))))

def fit_and_plot_dataset(concentrations, mean_response, sem_response, color, label, marker, ax, 
                        extrapolate_to=None, popt_for_extrapolation=None):
    plot_concentrations = np.array(concentrations)
    max_conc = 200 if extrapolate_to else max(plot_concentrations)
    X_smooth = np.logspace(np.log10(min(plot_concentrations)), np.log10(max_conc), 300)
    
    popt = None
    pcov = None
    y_smooth = None
    
    if popt_for_extrapolation is not None:
        popt = popt_for_extrapolation
        y_smooth = logistic(X_smooth, *popt)
    else:
        try:
            popt, pcov = curve_fit(logistic, plot_concentrations, mean_response,
                               p0=[100, 1, 10],
                               bounds=([0, -np.inf, 0], [200, np.inf, np.inf]))
            y_smooth = logistic(X_smooth, *popt)
        except Exception as e:
            print(f"Fitting failed for {label}: {e}")
    
    if y_smooth is not None:
        ax.plot(X_smooth, y_smooth, '-', color=color, linewidth=FITTED_CURVE_LINEWIDTH)
    
    ax.plot(plot_concentrations, mean_response, marker, color=color, 
            markersize=MARKER_SIZE, label=label, linestyle='None', alpha=0.8)
    
    ax.errorbar(plot_concentrations, mean_response, yerr=sem_response,
                fmt='none', ecolor=color, capsize=ERROR_BAR_CAPSIZE, 
                capthick=ERROR_BAR_THICKNESS, alpha=0.8)
    
    return popt, pcov

def calculate_ic50(popt, pcov):
    if popt is not None and pcov is not None:
        L, k, x0 = popt
        ic50 = x0
        perr = np.sqrt(np.diag(pcov))
        ic50_error = perr[2]
        return ic50, ic50_error, L, k, x0
    return None, None, None, None, None

df_siha = pd.read_csv("/workspaces/hpv_e6_hect_binding_inhibitors/MTT_C33A.csv")
df_c33a = pd.read_csv("/workspaces/hpv_e6_hect_binding_inhibitors/MTT_SiHa.csv")

if LAYOUT == 'horizontal':
    fig, axes = plt.subplots(1, 3, figsize=FIGURE_SIZE)
else:
    fig, axes = plt.subplots(3, 1, figsize=FIGURE_SIZE)

if not hasattr(axes, '__len__'):
    axes = [axes]

concentrations = [100, 50, 25, 12.5, 6.25, 3.125, 1.5625]
ticks = [1, 3, 10, 30, 100, 300]

print("Processing Panel A - IK01 data...")
ax = axes[0]

siha_ik01 = df_siha[df_siha['Compound'] == 'IK01']
c33a_ik01 = df_c33a[df_c33a['Compound'] == 'IK01']

siha_ik01_mean = siha_ik01['Mean'].values
siha_ik01_sem = siha_ik01['SEM'].values
c33a_ik01_mean = c33a_ik01['Mean'].values
c33a_ik01_sem = c33a_ik01['SEM'].values
c33a_ik01_conc = c33a_ik01['Concentration_uM'].values

ax.set_ylim(0, 120)
ax.set_xscale('log')
ax.set_xlim(10**0, 10**2.5)
ax.set_xlabel('IK01 Concentration (μM)', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.set_ylabel('Percent cell viability', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.text(PANEL_LABEL_X_OFFSET, PANEL_LABEL_Y_OFFSET, 'A', 
        transform=ax.transAxes, fontsize=PANEL_LABEL_FONTSIZE, fontweight='bold', va='top')
ax.set_xticks(ticks)
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)

siha_ik01_popt, siha_ik01_pcov = curve_fit(logistic, concentrations, siha_ik01_mean,
                              p0=[100, 1, 10],
                              bounds=([0, -np.inf, 0], [200, np.inf, np.inf]))

fit_and_plot_dataset(concentrations, siha_ik01_mean, siha_ik01_sem, '#f3d05b', 'SiHa IK01', 'o', ax,
                     extrapolate_to=200, popt_for_extrapolation=siha_ik01_popt)

c33a_ik01_popt, c33a_ik01_pcov = fit_and_plot_dataset(c33a_ik01_conc, c33a_ik01_mean, c33a_ik01_sem, "#706e70", 'C33A IK01', 'x', ax)

ax.legend(loc='upper right', fontsize=LEGEND_FONTSIZE)

print("Processing Panel B - IK02 data...")
ax = axes[1]

siha_ik02 = df_siha[df_siha['Compound'] == 'IK02']
c33a_ik02 = df_c33a[df_c33a['Compound'] == 'IK02']

siha_ik02_mean = siha_ik02['Mean'].values
siha_ik02_sem = siha_ik02['SEM'].values
c33a_ik02_mean = c33a_ik02['Mean'].values
c33a_ik02_sem = c33a_ik02['SEM'].values
c33a_ik02_conc = c33a_ik02['Concentration_uM'].values

ax.set_ylim(0, 120)
ax.set_xscale('log')
ax.set_xlim(10**0, 10**2.5)
ax.set_xlabel('IK02 Concentration (μM)', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.set_ylabel('', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.text(PANEL_LABEL_X_OFFSET, PANEL_LABEL_Y_OFFSET, 'B', 
        transform=ax.transAxes, fontsize=PANEL_LABEL_FONTSIZE, fontweight='bold', va='top')
ax.set_xticks(ticks)
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)

siha_ik02_popt, siha_ik02_pcov = curve_fit(logistic, concentrations, siha_ik02_mean,
                              p0=[100, 1, 10],
                              bounds=([0, -np.inf, 0], [200, np.inf, np.inf]))

fit_and_plot_dataset(concentrations, siha_ik02_mean, siha_ik02_sem, '#ec84a7', 'SiHa IK02', 'o', ax,
                     extrapolate_to=200, popt_for_extrapolation=siha_ik02_popt)

c33a_ik02_popt, c33a_ik02_pcov = fit_and_plot_dataset(c33a_ik02_conc, c33a_ik02_mean, c33a_ik02_sem, "#706e70", 'C33A IK02', 'x', ax)

ax.legend(loc='upper right', fontsize=LEGEND_FONTSIZE)

print("Processing Panel C - IK03 data...")
ax = axes[2]

siha_ik03 = df_siha[df_siha['Compound'] == 'IK03']
c33a_ik03 = df_c33a[df_c33a['Compound'] == 'IK03']

siha_ik03_mean = siha_ik03['Mean'].values
siha_ik03_sem = siha_ik03['SEM'].values
siha_ik03_conc = siha_ik03['Concentration_uM'].values
c33a_ik03_mean = c33a_ik03['Mean'].values
c33a_ik03_sem = c33a_ik03['SEM'].values
c33a_ik03_conc = c33a_ik03['Concentration_uM'].values

ax.set_ylim(0, 120)
ax.set_xscale('log')
ax.set_xlim(10**0, 10**2.5)
ax.set_xlabel('IK03 Concentration (μM)', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.set_ylabel('', fontsize=AXIS_LABEL_FONTSIZE, fontweight='bold')
ax.text(PANEL_LABEL_X_OFFSET, PANEL_LABEL_Y_OFFSET, 'C', 
        transform=ax.transAxes, fontsize=PANEL_LABEL_FONTSIZE, fontweight='bold', va='top')
ax.set_xticks(ticks)
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)

siha_ik03_popt, siha_ik03_pcov = fit_and_plot_dataset(siha_ik03_conc, siha_ik03_mean, siha_ik03_sem, 'teal', 'SiHa IK03', 'o', ax)
c33a_ik03_popt, c33a_ik03_pcov = fit_and_plot_dataset(c33a_ik03_conc, c33a_ik03_mean, c33a_ik03_sem, "#706e70", 'C33A IK03', 'x', ax)

ax.legend(loc='upper right', fontsize=LEGEND_FONTSIZE)

plt.tight_layout()

#print(f"\nSaving figures...")
plt.savefig(OUTPUT_PNG, dpi=SAVE_DPI, bbox_inches='tight')
plt.savefig(OUTPUT_PDF, dpi=SAVE_DPI, bbox_inches='tight')
print(f"Saved as: {OUTPUT_PNG} and {OUTPUT_PDF}")

plt.show()

